In [1]:

from katana import remote
from katana.remote import import_data

my_client = remote.Client()

print(my_client)


In [2]:

NUM_PARTITIONS  = 3
   #
DB_NAME         = "my_db"
GRAPH_NAME      = "my_graph"

print("--")


--


In [3]:

#  CONNECT TO GRAPH

my_graph, *_ = my_client.get_database(name=DB_NAME).find_graphs_by_name(GRAPH_NAME)

print(my_graph)


<_Graph my_graph, 5sence62JumWdFFiRo2bUE8yBTTbp3VShtTTK44K4YW7, 1>


In [4]:
display(my_graph.num_nodes())
display(my_graph.num_edges())

          0/? [?op/s]

29946

          0/? [?op/s]

43072

In [ ]:
# ##################################################################

In [ ]:
#  Making a GS Bucket Public,
#
#  From,
#     https://cloud.google.com/storage/docs/access-control/making-data-public#command-line

gsutil iam ch allUsers:objectViewer gs://farrell-bucket



In [ ]:
#  BEGIN: READ CSV, OUTPUT PARQUET, PROCESS SAME

import os
   #
import dask.dataframe as dd
import numpy as np
   #
from google.cloud import storage
import gcsfs


os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/home/jovyan/work/02_MyKeys.json"


l_bucket    = "katana-demo-datasets"
l_fullpath  = "gs://" + l_bucket + "/csv-datasets/ldbc/sf-0.003/csv/bi/composite-projected-fk/initial_snapshot/static/Place/part-00000-c16729dd-16d5-456b-84ca-d0f19fa783d6-c000.csv"


l_dataframe = dd.read_csv(
   l_fullpath,
   delimiter='|',
   dtype={
      'id':   int,
      "name": np.dtype('O'),
      "url":  np.dtype('O'),
      "type": np.dtype('O'),
      })


l_cntr = 0
   #
for l_each in l_dataframe.iterrows():
   l_cntr+=1
      #
   if (l_cntr < 10):
      print(l_each)
      print("")  
    

In [ ]:
#  OUTPUT THE ABOVE TO PARQUET
#
#     There was a lot of issue here with keys files, persmissions, other
#     See,
#        https://cloud.google.com/iam/docs/creating-managing-service-account-keys#iam-service-account-keys-list-gcloud
#
#        gcloud iam service-accounts keys list \
#           --iam-account=farrell@katana-internal39.iam.gserviceaccount.com


l_outputfolder = "gs://" + "farrell-bucket" + "/test"


l_dataframe.to_parquet(l_outputfolder,
   write_index         = False,
   write_metadata_file = False,
      #
   engine              = "pyarrow",
   version             = "2.6"
   )

print("--")


In [ ]:
#  READING A PARQUET FILE

l_df = dd.read_parquet("https://storage.googleapis.com/farrell-bucket/test/part.0.parquet")

l_df.head(5)


In [ ]:
# ##################################################################

In [ ]:
#  IMPORT DATA FROM PARQUET TO GRAPH

from katana.remote import import_data

l_node ="gs://farrell-bucket/test/50_NodeHeader.txt"
l_data ="gs://farrell-bucket"

import_data.parquet(
   my_graph,
      #
   input_node_path   = l_node,
   #  input_edge_path   = l_edge,
   input_dir         = l_data,
      #
   schema_delimiter="|",
   ids_are_integers=True
   )

print("--")


#  Use counters above
